In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv("ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
links = pd.read_csv("links.csv")
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies = pd.read_csv("movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
tags = pd.read_csv("tags.csv")
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [6]:
import sqlite3

In [7]:
conn=sqlite3.connect("dataset.db")
cur=conn.cursor()

In [10]:
ratings.to_sql("ratings", conn, if_exists="replace", index=False)

In [16]:
tags.to_sql("tags", conn, if_exists="replace", index=False)

In [17]:
movies.to_sql("movies", conn, if_exists="replace", index=False)

In [18]:
links.to_sql("links", conn, if_exists="replace", index=False)

In [19]:
def table_info(conn,cursor):
    """
    prints out all of the columns of every table in the DB
    
    conn: database connection object
    cursor: cursor object
    """
    
    tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    for table_name in tables:
        table_name = table_name[0]
        table = pd.read_sql_query("SELECT * from {} LIMIT 0".format(table_name), conn)
        print(table_name)
        for col in table.columns:
            print("\t "+ col)
            print()

In [20]:
table_info(conn,cur)

ratings
	 userId

	 movieId

	 rating

	 timestamp

tags
	 userId

	 movieId

	 tag

	 timestamp

movies
	 movieId

	 title

	 genres

links
	 movieId

	 imdbId

	 tmdbId



In [13]:
cur.execute("SELECT * FROM ratings")
cur.fetchone()

(1, 1, 4.0, 964982703)

In [15]:
cur.execute("SELECT COUNT(DISTINCT userId) FROM RATINGS")
cur.fetchall()

[(610,)]

In [26]:
cur.execute("SELECT COUNT(*) FROM ratings")
cur.fetchall()

[(100836,)]

In [27]:
cur.execute("SELECT COUNT(*) FROM MOVIES")
cur.fetchall()

[(9742,)]

In [58]:
dfs_to_join = [ratings, tags, movies, links]

In [47]:
dfs_to_join 

[        userId  movieId  rating   timestamp
 0            1        1     4.0   964982703
 1            1        3     4.0   964981247
 2            1        6     4.0   964982224
 3            1       47     5.0   964983815
 4            1       50     5.0   964982931
 ...        ...      ...     ...         ...
 100831     610   166534     4.0  1493848402
 100832     610   168248     5.0  1493850091
 100833     610   168250     5.0  1494273047
 100834     610   168252     5.0  1493846352
 100835     610   170875     3.0  1493846415
 
 [100836 rows x 4 columns],
       userId  movieId               tag   timestamp
 0          2    60756             funny  1445714994
 1          2    60756   Highly quotable  1445714996
 2          2    60756      will ferrell  1445714992
 3          2    89774      Boxing story  1445715207
 4          2    89774               MMA  1445715200
 ...      ...      ...               ...         ...
 3678     606     7382         for katie  1171234019
 3679 

In [56]:
dataset_all = pd.concat(dfs_to_join, join="inner", axis=1)

In [57]:
dataset_all.head()

,userId,movieId,rating,timestamp,userId,movieId,tag,timestamp,movieId,title,genres,movieId,imdbId,tmdbId
0,1,1,4.0,964982703,2,60756,funny,1445714994,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,114709,862.0
1,1,3,4.0,964981247,2,60756,Highly quotable,1445714996,2,Jumanji (1995),Adventure|Children|Fantasy,2,113497,8844.0
2,1,6,4.0,964982224,2,60756,will ferrell,1445714992,3,Grumpier Old Men (1995),Comedy|Romance,3,113228,15602.0
3,1,47,5.0,964983815,2,89774,Boxing story,1445715207,4,Waiting to Exhale (1995),Comedy|Drama|Romance,4,114885,31357.0
4,1,50,5.0,964982931,2,89774,MMA,1445715200,5,Father of the Bride Part II (1995),Comedy,5,113041,11862.0


In [88]:
dataset_all.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
userId         int64
movieId        int64
tag           object
timestamp      int64
movieId        int64
title         object
genres        object
movieId        int64
imdbId         int64
tmdbId       float64
dtype: object

In [92]:
dataset_all.isnull().sum(axis=0)

userId       0
movieId      0
rating       0
timestamp    0
userId       0
movieId      0
tag          0
timestamp    0
movieId      0
title        0
genres       0
movieId      0
imdbId       0
tmdbId       4
dtype: int64

In [93]:
dataset_all.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
userId         int64
movieId        int64
tag           object
timestamp      int64
movieId        int64
title         object
genres        object
movieId        int64
imdbId         int64
tmdbId       float64
dtype: object

In [96]:
cur.execute("PRAGMA table_info('dataset_all')").fetchall()

[]